This notebook shows you how to create and query a table or DataFrame loaded from data stored in Azure Blob storage.

### Step 1: Set the data location and type

There are two ways to access Azure Blob storage: account keys and shared access signatures (SAS).

To get started, we need to set the location and type of the file.

In [3]:
storage_account_name = "taxistorage2019"
storage_account_access_key = "/tOx7RcMAngv1Tnn4MbOAcaLFlAUS7DXBO+7nOvfc0SbNCks2mo6uZ2j3F+U/89iQnyTD3t8ScJeLt2eMkX1wA=="

In [4]:
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

### Step 2: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [6]:
dfTaxi = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("wasbs://taxicab@taxistorage2019.blob.core.windows.net/train.csv")

###Step 3: Save the Data

We save two different copies of the data for comparison of speed to run queries: One in csv, avro, and parquet.

In [8]:
dfTaxi.write.mode("overwrite").option("header", "true").format("csv").saveAsTable("TaxiCSV")

In [9]:
dfTaxi.write.mode("overwrite").option("header", "true").format("parquet").saveAsTable("TaxiParquet")

In [10]:
dfTaxi.write.mode("overwrite").option("header", "true").format("orc").saveAsTable("TaxiORC")

We can query this view using Spark SQL. For instance, we can perform a simple aggregation. Notice how we can use `%sql` to query the view from SQL.

In [12]:
%sql

SELECT COUNT(*) from TaxiCSV

count(1)
1710670


In [13]:
%sql
SELECT COUNT(*) FROM TaxiParquet

count(1)
1710670


In [14]:
%sql
SELECT COUNT(*) FROM TaxiORC

count(1)
1710670


###More complex calls

In [16]:
%sql
SELECT SUM(Case when DAY_TYPE = 'B' then 1 else 0 end) as Trips_On_Holiday
      , SUM(Case when DAY_TYPE = 'C' then 1 else 0 end) as Trips_Before_Holiday
      , SUM(Case when DAY_TYPE = 'A' then 1 else 0 end) as Trips_On_NormalDays
FROM TaxiCSV

Trips_On_Holiday,Trips_Before_Holiday,Trips_On_NormalDays
0,0,1710670


In [17]:
%sql
SELECT SUM(Case when DAY_TYPE = 'B' then 1 else 0 end) as Trips_On_Holiday
      , SUM(Case when DAY_TYPE = 'C' then 1 else 0 end) as Trips_Before_Holiday
      , SUM(Case when DAY_TYPE = 'A' then 1 else 0 end) as Trips_On_NormalDays
FROM taxiparquet

Trips_On_Holiday,Trips_Before_Holiday,Trips_On_NormalDays
0,0,1710670


In [18]:
%sql
SELECT SUM(Case when DAY_TYPE = 'B' then 1 else 0 end) as Trips_On_Holiday
      , SUM(Case when DAY_TYPE = 'C' then 1 else 0 end) as Trips_Before_Holiday
      , SUM(Case when DAY_TYPE = 'A' then 1 else 0 end) as Trips_On_NormalDays
FROM taxiorc

Trips_On_Holiday,Trips_Before_Holiday,Trips_On_NormalDays
0,0,1710670


In [19]:
%sql
SELECT SUM(Case when MISSING_DATA = false then 1 else 0 end) as No_Missing_Data
      , SUM(Case when MISSING_DATA = true then 1 else 0 end) as Missing_Data
FROM taxicsv

No_Missing_Data,Missing_Data
1710660,10


In [20]:
%sql
SELECT SUM(Case when MISSING_DATA = false then 1 else 0 end) as No_Missing_Data
      , SUM(Case when MISSING_DATA = true then 1 else 0 end) as Missing_Data
FROM taxiparquet

No_Missing_Data,Missing_Data
1710660,10


In [21]:
%sql
SELECT SUM(Case when MISSING_DATA = false then 1 else 0 end) as No_Missing_Data
      , SUM(Case when MISSING_DATA = true then 1 else 0 end) as Missing_Data
FROM taxiorc

No_Missing_Data,Missing_Data
1710660,10


Since this table is registered as a temp view, it will be available only to this notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.

In [23]:
df.write.format("parquet").saveAsTable("MY_PERMANENT_TABLE_NAME")

This table will persist across cluster restarts and allow various users across different notebooks to query this data.